# Face mask detection


In [ ]:
!pip install xmltodict --upgrade  # this will install xmltodict library

# importing other libraries
import cv2
import os
import random
import xmltodict
from PIL import Image
import numpy as np
import collections
import matplotlib.pyplot as plt
import lxml.etree

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
import seaborn as sns
from sklearn.utils import shuffle

# Exploratory Data Analysis

Let us see what data our dataset has





In [ ]:
# this will print a sample image from our dataset
img =cv2.imread("../input/face-mask-detection/images/maksssksksss244.png")   #reading the image 
out_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)   #converting it from RGB to BGR

plt.figure(figsize=(12,12))    #specifying the size of image to be printed
plt.imshow(out_img)     # ptinting the image

In [ ]:
#printing the annotations file of the above image
tree = lxml.etree.parse("../input/face-mask-detection/annotations/maksssksksss244.xml")
pretty = lxml.etree.tostring(tree, encoding="unicode", pretty_print=True)
print(pretty)

In [ ]:
# this function helps to extract the coordinates of the rectangles that bound the faces in these images
labels=[]
def face_exctraction(directory_load_image,directory_load_notation):
    
    
   #opening the annotations file in readable format so that useful data can be extracted from it
        with open(os.path.join(directory_load_notation,"maksssksksss244.xml" ), 'r') as f:
            data = f.read()


            #getting data
            ann_dict=xmltodict.parse(data)["annotation"]
            object_list = ann_dict["object"]
            file="maksssksksss244.xml"


            #checking if there are more than 1 face in this image
            if isinstance(object_list, list): #if yes: itrating through them 
                 for object_ in object_list:
                        #getting face boundaries
                        xmin = int(object_["bndbox"]["xmin"])
                        xmax = int(object_["bndbox"]["xmax"])
                        ymin = int(object_["bndbox"]["ymin"])
                        ymax = int(object_["bndbox"]["ymax"])
                        
                        cv2.rectangle(out_img,(xmin,ymin),(xmax,ymax),(175,200,150),2)#printing a rectangle around the face
                        labels.append(object_["name"])
        #printing the  new image
        
        plt.figure(figsize=(12,12))
        plt.imshow(out_img)   
        return labels
# executing the function  
labels=face_exctraction("../input/face-mask-detection/images/","../input/face-mask-detection/annotations/")

for labels in labels:
    print(labels)

In [ ]:
def face_exctraction(directory_load_image,directory_load_notation):
    labels = []
    
    
  
    
    for file in os.listdir(directory_load_notation):
        with open(os.path.join(directory_load_notation, file), 'r') as f:
            data = f.read()


            #getting data
            ann_dict=xmltodict.parse(data)["annotation"]
            object_list = ann_dict["object"]


        

            #checking if there are more than 1 face in this image
            if isinstance(object_list, list): #if yes: itrating through them 
                 for object_ in object_list:
                        #getting face boundaries
                        xmin = int(object_["bndbox"]["xmin"])
                        xmax = int(object_["bndbox"]["xmax"])
                        ymin = int(object_["bndbox"]["ymin"])
                        ymax = int(object_["bndbox"]["ymax"])
                        
                        labels.append(object_["name"])
                        
                           

                       
            else:
                #same thing on images with one face in them
                object_ = object_list
                xmin = int(object_["bndbox"]["xmin"])
                xmax = int(object_["bndbox"]["xmax"])
                ymin = int(object_["bndbox"]["ymin"])
                ymax = int(object_["bndbox"]["ymax"])

                labels.append(object_["name"])
               
                
    labels = np.asarray(labels)
    
    return labels
labels = face_exctraction("../input/face-mask-detection/images/","../input/face-mask-detection/annotations/")
print(labels.shape)
print(collections.Counter(labels))


In [ ]:
# this is a similar function 
#it helps to extract the cropped images of the faces of people in the images
# we will later pass these cropped images to the model for training it
def face_exctraction(directory_load_image,directory_load_notation):
        faces=[] #this will store the cropped images
        labels=[]
   
        with open(os.path.join(directory_load_notation,"maksssksksss244.xml" ), 'r') as f:
            data = f.read()


            #getting data
            ann_dict=xmltodict.parse(data)["annotation"]
            object_list = ann_dict["object"]
            file="maksssksksss244.xml"

            #opening image
            image_name = file.replace("xml","png")
            im = Image.open(directory_load_image+image_name)
            print(type(im))

            #checking if there are more than 1 face in this image
            if isinstance(object_list, list): #if yes: itrating through them 
                 for object_ in object_list:
                        #getting face boundaries
                        xmin = int(object_["bndbox"]["xmin"])
                        xmax = int(object_["bndbox"]["xmax"])
                        ymin = int(object_["bndbox"]["ymin"])
                        ymax = int(object_["bndbox"]["ymax"])
                        crop_rectangle = (xmin, ymin, xmax , ymax) 
                        cropped_im = im.crop(crop_rectangle)  # cropping faces from the given image
                        faces.append(cropped_im)              # saving cropped images in the list
                        labels.append(object_["name"])
        return faces,labels
# executing the function      
faces,labels=face_exctraction("../input/face-mask-detection/images/","../input/face-mask-detection/annotations/")

print(plt.imshow(faces[1]))   #printing the images
print(labels[1])  

# Data pre-processing


In [ ]:
# this function saves the cropped images of faces in two folders(train_images and test_images)
# it also saves the labels("with_mask" or "without_mask") of these cropped images in two lists y_train and y_test
# we are splitting our data in to train and test. one for training our model and other for testing our model
# it will also resize our images to (100,100) as all images are of different sizes so to maintain uniformity in our data it will resize the images
def face_exctraction(directory_load_image,directory_load_notation,directory_save_train,directory_save_test):
    y_train = []
    y_test = []
    i=1
    
    if not os.path.exists("./"+directory_save_train):
        os.makedirs("./"+directory_save_train)
    if not os.path.exists("./"+directory_save_test):
        os.makedirs('./'+directory_save_test)
    
    for file in os.listdir(directory_load_notation):
        with open(os.path.join(directory_load_notation, file), 'r') as f:
            data = f.read()


            #getting data
            ann_dict=xmltodict.parse(data)["annotation"]
            object_list = ann_dict["object"]


            #opening image
            image_name = file.replace("xml","png")
            im = Image.open(directory_load_image+image_name)

            #checking if there are more than 1 face in this image
            if isinstance(object_list, list): #if yes: itrating through them 
                 for object_ in object_list:
                        #getting face boundaries
                        xmin = int(object_["bndbox"]["xmin"])
                        xmax = int(object_["bndbox"]["xmax"])
                        ymin = int(object_["bndbox"]["ymin"])
                        ymax = int(object_["bndbox"]["ymax"])
                        
                        #cropping the face and saving it
                        crop_rectangle = (xmin, ymin, xmax , ymax)
                        cropped_im = im.crop(crop_rectangle)
                        resized_im = cropped_im.resize((100,100), Image.ANTIALIAS)
                        
                        #this part is for sperating train set from test set
                        if i<3600:
                            resized_im.save(directory_save_train+str(i)+".png")
                            y_train.append(object_["name"])
                        else:
                            resized_im.save(directory_save_test+str(i-3599)+".png")
                            y_test.append(object_["name"])

                        i+=1
            else:
                #same thing on images with one face in them
                object_ = object_list
                xmin = int(object_["bndbox"]["xmin"])
                xmax = int(object_["bndbox"]["xmax"])
                ymin = int(object_["bndbox"]["ymin"])
                ymax = int(object_["bndbox"]["ymax"])

                crop_rectangle = (xmin, ymin, xmax , ymax)
                cropped_im = im.crop(crop_rectangle)
                resized_im = cropped_im.resize((100, 100), Image.ANTIALIAS)
                if i<3600:
                    resized_im.save(directory_save_train+str(i)+".png")
                    y_train.append(object_["name"])
                else:
                    resized_im.save(directory_save_test+str(i-3599)+".png")
                    y_test.append(object_["name"])

                i+=1
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    return y_train,y_test


In [ ]:
# executing the previous function
y_train,y_test = face_exctraction("../input/face-mask-detection/images/","../input/face-mask-detection/annotations/","train_images/","test_images/")

In [ ]:
# it will count the number of items in our list
print(collections.Counter(y_train))
print(collections.Counter(y_test))

In [ ]:
# we can not pass images directly to our model so here we will convert our images into  numpy arrays
def images_to_array(directory,num_img):
    X_nn=[]
    #iterating through directory and turning 100x100 images to an array with shape of (100,100,3) and stacking them inside a dataset.
    for i in range(1,num_img+1):
        im = Image.open(directory+str(i)+".png")
        rgb_im = im.convert('RGB')
        im = np.asarray(rgb_im,dtype="int")
        X_nn.append(im)
    return np.asarray(X_nn)

In [ ]:
#executing the previous function
X_train = images_to_array("./train_images/",3599)
X_test = images_to_array("./test_images/",473)
#printing the shape of our saved data
X_train.shape
#it will print (3599, 100, 100, 3)
#here 3599 is the number of images in X_train
# 100x100 is the size of image
# 3 is the number of colour channels (RGB - RED GREEN BLUE)

In [ ]:
# so now we have arrays of images and their labels
# example
print("label= ",y_train[0]) # this prints the label of first image
print()
print("Array = ",X_train[0]) #this prints the numpy array of first image

**Generating new images**

In [ ]:

# it will generate new images from the provided images to increase the  amount of traning data  
def add_data_aug (directory_save_inc , directory_save_no , num_inc , num_no , X_train , y_train):
    
    if not os.path.exists("./"+directory_save_inc):
        os.makedirs("./"+directory_save_inc)
    if not os.path.exists("./"+directory_save_no):
        os.makedirs('./'+directory_save_no)
    
    
    #the image generator ranges.
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = "nearest"
    )
    
    #we are trying to upsmaple "without_mask" images and "mask_weared_incorrect" so we select them
    without_mask = X_train[y_train=="without_mask"]
    mask_weared_incorrect = X_train[y_train=="mask_weared_incorrect"]
    
    #creating the desired number of images
    j=0
    for batch in datagen.flow(without_mask,batch_size=1,save_to_dir=directory_save_no,save_prefix="wo",save_format='png'):
        j+=1
        #if j> 2175:
        if j>num_no:
            break
    j=0
    for batch in datagen.flow(mask_weared_incorrect,batch_size=1,save_to_dir=directory_save_inc,save_prefix="wi",save_format='png'):
        j+=1
        #if j> 2722:
        if j>num_inc:
            break
    X_nn=[]
    
    #adding newly made images to trainign set
    for file in os.listdir(directory_save_no):
        im = Image.open(os.path.join(directory_save_no, file))
        rgb_im = im.convert('RGB')
        im = np.asarray(rgb_im,dtype="int")
        X_nn.append(im)
        y_train=np.append(y_train,'without_mask')
    
    
    for file in os.listdir(directory_save_inc):
        im = Image.open(os.path.join(directory_save_inc, file))
        rgb_im = im.convert('RGB')
        im = np.asarray(rgb_im,dtype="int")
        X_nn.append(im)
        y_train = np.append(y_train,"mask_weared_incorrect")
        
    X_train_augmented = np.asarray(X_nn)
    
    X_train = np.concatenate((X_train,X_train_augmented  ))
    
    X_train, y_train = shuffle(X_train, y_train, random_state=0)
    
    return X_train,y_train

In [ ]:
#executing the previous function
X_train,y_train = add_data_aug ("Augmented_incorrect/" , "Augmented_without/", 2722 , 2175 , X_train , y_train)

In [ ]:
#see the increased number of images
print(collections.Counter(y_train))
print(y_train.shape)

In [ ]:
def preproccesing(X_train,X_test,y_train,y_test):
    #nomalizing color values
    X_train = X_train / 255
    X_test = X_test / 255
    
    #categorical to numerical values for labels.
    conditions = [
                    (y_train == "without_mask"),
                    y_train == ("mask_weared_incorrect"),
                    (y_train == "with_mask")]
    choices = [0, 1, 2]
    y_train = np.select(conditions, choices)
    conditions = [
                    (y_test == "without_mask"),
                    y_test == ("mask_weared_incorrect"),
                    (y_test == "with_mask")]
    choices = [0, 1, 2]
    y_test = np.select(conditions, choices)
    return X_train,X_test,y_train,y_test

In [ ]:
#executing the function
X_train,X_test,y_train,y_test = preproccesing(X_train,X_test,y_train,y_test)

In [ ]:
collections.Counter(y_train)

****So now we have the two training and 2 testing arrays of images and labels****

# Model

**Downloading pre trained model**

In [ ]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(100,100,3)))
#we will remove the top layer of the model 
print(baseModel.summary())


**Adding our own classification layers**

In [ ]:
headModel=baseModel.output #taking the output of base model
headModel=AveragePooling2D(pool_size=(4,4))(headModel) #doing average pooling of 2D 7x7 matrix
headModel=Flatten(name='Flatten')(headModel) #flattening the layers
headModel=Dense(128,activation='relu')(headModel) # Adding 128 dense layers
headModel=Dropout(0.7)(headModel) # adding droupout layers to prevent overfitting
headModel=Dense(3,activation='softmax')(headModel) #last dense layers for our 3 classes

model=Model(inputs=baseModel.input,outputs=headModel) 

In [ ]:
# making the basemodel layers non trainable
for layer in baseModel.layers:
    layer.trainable=False


print(model.summary())

**Adding checkpoints to save best model**

In [ ]:
#adding checkpoints so that we can save the best model from all
checkpoint = ModelCheckpoint('best_model.h5',  # model filename
                             monitor='val_loss', # quantity to monitor
                             verbose=0, # verbosity - 0 or 1
                             save_best_only= True, # The latest best model will not be overwritten
                             mode='auto') # The decision to overwrite model is made 
                                          # automatically depending on the quantity to monitor

**Compiling the model**

In [ ]:
learning_rate=0.0001
Epochs=50
BS=30

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)

#compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
#starting the training of model
H=model.fit (
   X_train,y_train,batch_size=BS,
    steps_per_epoch=len(X_train)//BS,
    validation_data=(X_test,y_test),
    validation_steps=len(X_test)//BS,
    epochs=Epochs,
    callbacks = [checkpoint],
    verbose=1
)

**Plotting the accuracy/loss vs epochs graph**

In [ ]:
#plotting the accuracy and loss of the model
N = Epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('a.png')

**Evaluating the model using classification report and confusion matrix**

In [ ]:
def evaluatuion (model,X_test,y_test):
    #getting models predcitons on test set
    y_pred = []
    y_prob = model.predict(X_test)
    for i in y_prob:
        y_pred.append(np.argmax(i))
        
                
    confusion_matrix_array = confusion_matrix(y_test,y_pred)
    print("simple confusion matrix")
    print(confusion_matrix_array)
    print("Report:")
    print(classification_report(y_test, y_pred,
	target_names=["without mask","mask weared incorrectly","with mask"]))
    
    
    print("Confusion Matrix:")
    confusion_matrix_array = confusion_matrix(y_test,y_pred)
    
    g= sns.heatmap(confusion_matrix_array, annot=True,yticklabels=["Act No Mask","Act mask weared incorrectly","Act with mask"],xticklabels=["Pred No Mask","Pred mask weared incorrectly","Pred with mask"])
    g.set_yticklabels(g.get_yticklabels(), rotation = 0)
    

    

    
    return y_pred
    

In [ ]:
model=load_model("best_model.h5")
y_pred = evaluatuion (model,X_test,y_test,)

In [ ]:
model.save('best.model')

In [ ]:
model.save('best.keras')

In [ ]:
 cv2.putText(new_img,mask_label[mask_result.argmax()],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)